In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_name = './output/20200313_235302/20200313_235302.csv'
predict = pd.read_csv(file_name, usecols=['demand'])
predict.head()

In [ ]:
# マイナスを除く
predict['demand'] = predict['demand'].map(lambda x: np.maximum(0, x))
predict.head()

In [ ]:
predict[predict['demand'] < 0].count()

In [ ]:
# data = pd.read_pickle('./resources/feature/feature.pkl').reset_index()

# test = data[(data["date"] > "2016-04-24")].reset_index()
# test = test[['id', 'date', 'demand']]

test = pd.read_csv('./test.csv')
test.head()

In [ ]:
test['demand'] = predict['demand']
test.head()

In [ ]:
def make_submission(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)

In [ ]:
make_submission(test, pd.read_csv('../input/m5-forecasting-accuracy/sample_submission.csv'))

In [ ]:
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m "長い期間、Origin Feature追加"